In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import random
import time
pd.set_option('display.max_row',2000)
pd.set_option('display.max_columns',100)

pd.set_option('display.max_seq_items',None)
#df=pd.read_excel("/content/drive/MyDrive/toeic_word(2).xlsx",sheet_name="word",engine="openpyxl")
df=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TOEIC/toeic_word(basic).xlsx",sheet_name="word",engine="openpyxl")

#dfStyler = df.style.set_properties(**{'text-align': 'left'})
#dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

In [11]:
def get_key():
    key=input()
    return key

def simple_print(set_):
    num=1
    for i in set_:
        print(num)
        i.simple_view()
        num=num+1

def detail_print(set_):
    num=1
    for i in set_:
        print(num)
        i.details_view()
        num=num+1

def excel_print(set_):
    print(len(set_))
    for i in set_:
        i.excel_word_view()
    for i in set_:
        i.excel_mean_view()

class word:
    def __init__(self,word,mean,chapter_num,chapter_name,answer,wrong,recent_wrong,exam_rate):
        self.word=word
        self.mean=mean
        self.chapter_num=chapter_num
        self.chapter_name=chapter_name
        self.answer=answer
        self.wrong=wrong
        self.recent_wrong=recent_wrong
        self.exam_rate=exam_rate

    def __eq__(self,other):
        return self.word == other.word

    def __hash__(self):
        return hash(self.word)

    def modify_word(self,word):
        self.answer=word.answer
        self.wrong=word.wrong
        self.recent_wrong=word.recent_wrong

    def simple_view(self):
        print(self.word, self.mean)
    def details_view(self):
        print(self.word, str(self.mean), "(",self.chapter_name,",", str(self.chapter_num)+"Chapter ","answer: ",self.answer,", wrong: ",self.wrong,", recent_wrong: ",self.recent_wrong,")")
    def excel_word_view(self):
        print(self.word,"|")
    def excel_mean_view(self):
        print(self.mean,"|")
    def mean_test(self):
        print(self.word+"의 뜻은? ")
        key=get_key()
        return self.decide_answer(key)
    def decide_answer(self,key):
        while key == 'right ctrl':
            key=get_key()
        if key == 'up' or key == 'down':
            self.details_view()
            time.sleep(0.2)
            key=get_key()
            if key == 'right':
                return True
            else:
                return False
        elif key == 'right':
            self.details_view()
            return True
        else:
            self.details_view()
            return False
    def _wrong(self):
        self.recent_wrong=1
        self.wrong=self.wrong+1
    def _answer(self):
        self.recent_wrong=0
        self.answer=self.answer+1


class test_manager:
    def __init__(self):
        self.test_paper=list()

    def shuffle(self):
        random.shuffle(self.test_paper)

    def test_paper_set(self,num,set_):
        if num==0:
            self.test_paper=list(set_)
            return

        if(num<=len(set_)):
            list_=list(set_)
            count=0
            while True:
                if(count >= num):
                    break
                for i in list_:
                    if(count >= num):
                        break
                    if(random.random()<i.exam_rate):
                        self.test_paper.append(i)
                        list_.remove(i)
                        count = count + 1
    def exam(self,set_manager):
        answer_=0
        wrong_=0
        for i in self.test_paper:
            print(1+answer_+wrong_,".  ",end="")
            if i.mean_test():
                answer_ = answer_+1
                i._answer()
            else:
                wrong_ = wrong_ + 1
                i._wrong()
        print("정답: ",answer_,", 오답: ",wrong_)
        set_manager.result_SYN(self.test_paper)

    def simple_test(self):
        num=1
        for i in self.test_paper:
            print(num,".  ",i.word,"  ",end='')
            key = get_key()
            if i.decide_answer(key) or ~i.decide_answer(key):
                num = num+1

class set_manager:
    def __init__(self,df):
        self.original=set()
        self.duplicate=set()
        self.df=df
        self.set_dataframe(self.original,df)

    def chapter_select(self,*chapter_num):
        self.set_dataframe(self.duplicate,self.df[self.df['day'].isin(chapter_num)])

    def except_chapter_select(self,*chapter_num):
        self.set_dataframe(self.duplicate,self.df[~self.df['day'].isin(chapter_num)])

    def bigger_chapter_select(self,chapter_num):
        self.set_dataframe(self.duplicate,self.df[self.df['day'] > chapter_num])

    def smaller_chapter_select(self,chapter_num):
        self.set_dataframe(self.duplicate,self.df[self.df['day'] < chapter_num])

    def find_word(self,set_,word):
        for i in set_:
            if i==word:
                return i
        return false

    def set_dataframe(self,set_,df):
        set_.clear()
        for i in range(0,df.shape[0]):
            loc=df.iloc[i]
            temp=word(loc[1],loc[2],loc[4],loc[5],loc[6],loc[7],loc[8],loc[9])
            set_.add(temp)

    def result_SYN(self,set_):
        for i in set_:
            self.df.loc[self.df["영단어"]==i.word,"정답"]=i.answer
            self.df.loc[self.df["영단어"]==i.word,"오답"]=i.wrong
            self.df.loc[self.df["영단어"]==i.word,"최근오답여부"]=i.recent_wrong
        self.original.clear()
        self.duplicate.clear()
        self.df.to_excel('toeic word_data.xlsx', index=False, sheet_name="word")

In [12]:
test_set=set_manager(df)
#test_set.bigger_chapter_select(0)
test_set.chapter_select(29,30)
exam_=test_manager()
exam_.test_paper_set(0,test_set.duplicate)

In [ ]:
exam_.shuffle()
#detail_print(exam_.test_paper)
#simple_print(exam_.test_paper)
exam_.simple_test()
#exam_.exam(test_set)
#exam_.test_paper

1 .   operation   operation 수술 ( 건강 , 30Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
operation 수술 ( 건강 , 30Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
2 .   habitat   habitat (동, 식물의) 서식지 ( 환경 , 29Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
habitat (동, 식물의) 서식지 ( 환경 , 29Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
3 .   prescribe   prescribe (약을) 처방하다 ( 건강 , 30Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
prescribe (약을) 처방하다 ( 건강 , 30Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
4 .   pollutant   pollutant 오염 물질 ( 환경 , 29Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
pollutant 오염 물질 ( 환경 , 29Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
5 .   participate   participate 참여하다, 참가하다 ( 건강 , 30Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
participate 참여하다, 참가하다 ( 건강 , 30Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
6 .   dense   dense 빽빽한, 밀집한 ( 환경 , 29Chapter  answer:  0 , wrong:  0 , recent_wrong:  0 )
dense 빽빽한, 밀집